Library

In [2]:
import speech_recognition as sr
import difflib

from g2p_en import G2p
from colorama import Fore

Source

In [3]:
sentence = "london the capital city of the united kingdom is a vibrant metropolis rich in history and culture Known as the square mile the city of london is the historic core where the romans first established londinium today it's a major business and financial center housing the Bank of England the royal exchange and the london stock exchange despite its modern skyscrapers like the gherkin and the walkie talkie london retains its historical charm with landmarks such as tower of london the city's boundaries have remained nearly unchanged since medieval times making it a unique blend of ancient and contemporary with a small resident population but a bustling daytime workforce the city is always alive with activity reflecting its status as one of the world's leading financial hubs."

In [4]:
audio_file = "E:\Perkuliahan\Semester 4\Pembelajaran Mesin\Tugas Membuat Makalah\Speech-to-text\Data Suara\Data 8.wav"

Function to get input from Speech-to-Text

In [5]:
def get_speech_to_text(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language="en-us", show_all=False)
            return text.lower()
        except sr.UnknownValueError as e:
            print("Speech recognition could not understand audio")
            return ""
        except sr.RequestError as e:
            print("Error from Google Speech Recognition service")
            return ""

Function to convert Text to ARPAbet

In [6]:
g2p = G2p()
def text_to_arpabet(text):
    phonemes = g2p(text)
    return phonemes

Function to Compare word based on ARPAbet

In [7]:
def compare_pronunciations(word1, word2):
    phonemes1 = text_to_arpabet(word1)
    phonemes2 = text_to_arpabet(word2)
    return phonemes1 == phonemes2

Function Error Detection and Correction

First Methode (Error detection)

In [9]:
def detect_pronunciation_error(provided_sentences, user_sentences):
    refernce_words = provided_sentences.split()
    user_words = user_sentences.split()
    result = []
    for reference, user in zip(refernce_words, user_words):
        correct = compare_pronunciations(reference, user)
        if not correct:
            result.append({
                'ref' : Fore.WHITE + reference,
                'user' : Fore.RED + user,
                'Note' : False
            })
        else:
            result.append({
                'ref' : Fore.WHITE + reference,
                'user' : Fore.GREEN + user,
                'Note' : True
            })
    return result

Print the result

In [11]:
user_words = get_speech_to_text(audio_file)
results = detect_pronunciation_error(sentence, user_words)

for result in results:
    print(f"{result['ref']} <-> {result['user']}")

london <-> london
the <-> the
capital <-> capital
city <-> city
of <-> of
the <-> the
united <-> united
kingdom <-> kingdom
is <-> is
a <-> a
vibrant <-> fire
metropolis <-> metabolic
rich <-> rate
in <-> in
history <-> history
and <-> and
culture <-> culture
Known <-> know
as <-> as
the <-> the
square <-> square
mile <-> mile
the <-> the
city <-> city
of <-> of
london <-> london
is <-> is
the <-> the
historic <-> historical
core <-> where
where <-> the
the <-> romans
romans <-> first
first <-> established
established <-> london
londinium <-> you
today <-> today
it's <-> it's
a <-> a
major <-> major
business <-> business
and <-> and
financial <-> financial
center <-> center
housing <-> hosting
the <-> the
Bank <-> bank
of <-> of
England <-> england
the <-> the
royal <-> royal
exchange <-> exchange
and <-> and
the <-> london
london <-> stock
stock <-> exchange
exchange <-> despite
despite <-> his
its <-> mother
modern <-> and
skyscrapers <-> sky
like <-> characters
the <-> like
gherkin 

First Methode (Error correction)

In [18]:
Correct_words = []
Wrong_words = []
Error = 0
for result in results:
    Wrong_words.append(result['user'])
    if (result['Note'] == False):
        Correct_words.append(result['ref'])
        Error += 1
    else:
        Correct_words.append(result['user'])
print(Fore.WHITE + "Wrong Words : " + ' '.join(Wrong_words))
print(Fore.WHITE + "Corrected Words : " + ' '.join(Correct_words))
print("Refrence Words : ", sentence, '\n')
print('panjang reference : ', len(sentence), ' , panjang user_sentence : ', len(user_words))
print(f"Error Precentage from detection : {(Error/len(Correct_words)*100)} %")

Wrong Words : london the capital city of the united kingdom is a fire metabolic rate in history and culture know as the square mile the city of london is the historical where the romans first established london you today it's a major business and financial center hosting the bank of england the royal exchange and london stock exchange despite his mother and sky characters like the hurricane and the walking distance from tower of london the city missouri weather
Corrected Words : london the capital city of the united kingdom is a vibrant metropolis rich in history and culture Known as the square mile the city of london is the historic core where the romans first established londinium today it's a major business and financial center housing the bank of england the royal exchange and the london stock exchange despite its modern skyscrapers like the gherkin and the walkie talkie london retains its historical charm with landmarks such as
Refrence Words :  london the capital city of the unit

Second Methode (Function error detection and correction)

In [33]:
def Error_Detection_and_Correction(reference_sentence, user_sentence):
    reference_words = reference_sentence.split()
    user_words = user_sentence.split()

    difference = difflib.ndiff(user_words, reference_words)
    matcher = difflib.SequenceMatcher(None, user_words, reference_words)

    # Look for the difference between user words and reference words
    diff = []
    for word in difference:
        if word.startswith('- '):
            diff.append(f"Reference missing: {word[2:]}")
        elif word.startswith('+ '):
            diff.append(f"Extra in test: {word[2:]}")
        elif word.startswith('? '):
            pass  # This line shows markers for different characters, we can ignore it
    
    # Calculate Errors
    errors = 0
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag != 'equal':
            errors += max(i2 - i1, j2 - j1)

    # Correct the user sentence
    corrected_sentence = []
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            corrected_sentence.extend(user_words[i1:i2])
        elif tag == 'replace':
            corrected_sentence.extend(reference_words[j1:j2])
        elif tag == 'insert':
            corrected_sentence.extend(reference_words[j1:j2])
        elif tag == 'delete':
            continue

    return diff, errors, ' '.join(corrected_sentence)


Second Methode (Print Error Detection and Correction)

In [35]:
corrected_sentence = get_speech_to_text(audio_file)
Error_words, error, corrected_sentence = Error_Detection_and_Correction(sentence, corrected_sentence)
error_ratio = (error/len(sentence.split()))*100
# for word in Error_words:
#     print(word)
# print('\n')
print(f"Number of errors: {error_ratio} %")
print("Corrected sentence: \n", corrected_sentence)
print("Reference sentence: \n", sentence, "\n")

Number of errors: 58.26771653543307 %
Corrected sentence: 
 london the capital city of the united kingdom is a vibrant metropolis rich in history and culture Known as the square mile the city of london is the historic core where the romans first established londinium today it's a major business and financial center housing the Bank of England the royal exchange and the london stock exchange despite its modern skyscrapers like the gherkin and the walkie talkie london retains its historical charm with landmarks such as tower of london the city's boundaries have remained nearly unchanged since medieval times making it a unique blend of ancient and contemporary with a small resident population but a bustling daytime workforce the city is always alive with activity reflecting its status as one of the world's leading financial hubs.
Reference sentence: 
 london the capital city of the united kingdom is a vibrant metropolis rich in history and culture Known as the square mile the city of lond